In [1]:
# global var set
import transformers
# model info, change as needed
batch_size = 32
num_epochs = 21
model_checkpoint = "bert-base-uncased"
currentEmo = 'joy'

#fileTag = "clean-phase-noCombin-v1"
fileTag = 'original-plutchik-v1'

In [2]:
## garbage collection
import gc; gc.collect()
import torch
torch.cuda.empty_cache()

# Convert dataset to suitable format
IMPORTANT: please never run this section again if you have your dataset ready!

In [3]:
import pandas as pd
from pandas import DataFrame

trainDatasetOriginal = pd.read_csv(f'../../data/csv_version/dev/emotion/allcharlinepairs-{fileTag}.csv')
testDatasetOriginal = pd.read_csv(f'../../data/csv_version/test/emotion/allcharlinepairs-{fileTag}.csv')

In [4]:
import ast
trainDatasetProcessed = DataFrame({'sentence' : trainDatasetOriginal['sentence'], 
                                   'label' : [ast.literal_eval(trainDatasetOriginal['plutchik'][index])[currentEmo] for index in range(trainDatasetOriginal.shape[0])]})
testDatasetProcessed = DataFrame({'sentence' : testDatasetOriginal['sentence'], 
                                  'label' : [ast.literal_eval(testDatasetOriginal['plutchik'][index])[currentEmo] for index in range(testDatasetOriginal.shape[0])]})


In [5]:
trainDatasetProcessed.to_csv(f'./dataset/{currentEmo}-{fileTag}-train.csv')
trainDatasetProcessed

,sentence,label
0,I began making fish curry for my boyfriend and I.,3
1,I began making fish curry for my boyfriend and I.,2
2,I began making fish curry for my boyfriend and I.,2
3,I began making fish curry for my boyfriend and I.,0
4,I decided not to read a recipe since I've made...,2
...,...,...
53229,Suddenly he felt a sharp pinch.,0
53230,A crab was inside the shell pinching his leg.,0
53231,A crab was inside the shell pinching his leg.,0
53232,A crab was inside the shell pinching his leg.,0


In [6]:
testDatasetProcessed.to_csv(f'./dataset/{currentEmo}-{fileTag}-test.csv')
testDatasetProcessed

,sentence,label
0,A cook was carrying an armful of oranged in th...,3
1,A cook was carrying an armful of oranged in th...,2
2,He dropped one on the floor by accident.,0
3,He dropped one on the floor by accident.,0
4,He dropped one on the floor by accident.,0
...,...,...
51886,"After she stuffed herself, she regretted it.",0
51887,"After she stuffed herself, she regretted it.",0
51888,Rosemary swore to eat healthier tomorrow.,2
51889,Rosemary swore to eat healthier tomorrow.,0


# Start load

In [7]:
from datasets import load_dataset, load_metric

In [8]:
dataset = load_dataset('csv', data_files={'train': f'./dataset/{currentEmo}-{fileTag}-train.csv', 
                                           'test': f'./dataset/{currentEmo}-{fileTag}-test.csv'})

Using custom data configuration default-d7dc0881c27e3ed6


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\JAM_0\.cache\huggingface\datasets\csv\default-d7dc0881c27e3ed6\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
# global var set
import transformers
# model info, change as needed
batch_size = 32
num_epochs = 21
model_checkpoint = "bert-base-uncased"
currentEmo = 'joy'

#fileTag = "clean-phase-noCombin-v1"
fileTag = 'original-plutchik-v1'

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

In [11]:
tokenized = dataset.map(preprocess_function, batched=True)

  0%|          | 0/54 [00:00<?, ?ba/s]

  0%|          | 0/52 [00:00<?, ?ba/s]

In [12]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Trainer

In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [14]:
import datasets
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [15]:
# TO-DO
# 用于predict 8 个numbers
# 一个layer, dense layer
# multi regression
# MAE/MSE
# 

training_args = TrainingArguments(
    output_dir = './results',
    evaluation_strategy = "epoch",
    learning_rate = 5e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    weight_decay = 0.01,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, Unnamed: 0. If sentence, Unnamed: 0 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Python\miniconda3\envs\pytorchEnvWithDataSci\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 53234
  Num Epochs = 21
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 34944


RuntimeError: Found dtype Long but expected Float

In [12]:
import pandas as pd
dataLog = pd.DataFrame(trainer.state.log_history)
dataLog.to_csv(f'./trainingMetric/info-{currentEmo}-{fileTag}.csv')